### **一、码书模型（Code Book）**

#### **核心思想**

码书模型通过为**每个像素建立动态码本**来描述背景的复杂变化。每个码本包含多个码元（阈值范围），通过匹配当前像素值与历史码元来判断其属于背景还是前景。

- **码元更新**：若像素值匹配码元，则更新码元的阈值；若不匹配，则新增码元以应对动态背景（如树叶晃动、光照渐变）。
- **适应性**：适合处理多模态背景（如水面波纹、动态植被），但对剧烈变化的场景（如突然光照突变）鲁棒性较弱。

#### **技术分类**

- **背景差分法**：属于基于像素的背景建模方法，通过建立背景模型并与当前帧对比检测前景。

In [ ]:
# 安装 OpenCV（如果尚未安装）
%pip install opencv-python
import cv2
import numpy as np

# 初始化视频捕捉，替换为你的视频路径
cap = cv2.VideoCapture('video.mp4')  # 读取视频文件

# 获取原始视频的帧率和尺寸
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义输出视频的编码格式和文件名
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用 mp4v 编码格式
out = cv2.VideoWriter('output_foreground_mask.mp4', fourcc, fps, (frame_width, frame_height), isColor=False)

cv2.namedWindow('Original Video', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Original Video', 800, 600)  # 设置窗口大小为 800x600
cv2.namedWindow('Foreground Mask', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Foreground Mask', 800, 600)  # 设置窗口大小为 800x600

# 检查视频是否成功打开
if not cap.isOpened():
    print("错误：无法打开视频。")
else:
    # 读取第一帧
    ret, frame = cap.read()
    if ret:
        # 将第一帧转换为 HSV 颜色空间
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # 初始化码书，用于背景建模
        codebook = np.zeros_like(hsv, dtype=np.float32)

        # 主循环
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # 将当前帧转换为 HSV 并转为 float32 类型
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype(np.float32)

            # 更新码书（背景模型）
            codebook = cv2.addWeighted(codebook, 0.9, hsv, 0.1, 0)

            # 计算前景掩码
            fg_mask = cv2.absdiff(hsv, codebook)  # 计算帧与背景的差异
            fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_BGR2GRAY)  # 转为灰度图
            _, fg_mask = cv2.threshold(fg_mask, 50, 255, cv2.THRESH_BINARY)  # 二值化

            # 显示原视频
            cv2.imshow('Original Video', frame)
            # 显示前景掩码视频
            cv2.imshow('Foreground Mask', fg_mask)

            # 将前景掩码写入输出视频
            out.write(fg_mask)

            # 按 'q' 键退出
            if cv2.waitKey(30) & 0xFF == ord('q'):
                break

    # 释放资源并关闭窗口
    cap.release()
    out.release()  # 释放输出视频写入器
    cv2.destroyAllWindows()

应用场景与特点
应用：这段代码适用于检测视频中的动态目标，例如鱼在水中的移动（正如视频文件名“Fish Swimming Over Skip”所暗示的）。
特点：
使用HSV颜色空间，使系统对光照变化有一定的鲁棒性。
码书模型简单高效，适合实时处理。
通过调整cv2.addWeighted的权重（0.9和0.1）和cv2.threshold的阈值（50），可以优化检测效果。